In [11]:
# reload notebook
%load_ext autoreload
%autoreload 2
import jax 
import sys 
sys.path.append("/scr/aliang80/changepoint_aug/changepoint_aug/density_estimation")
import pickle
import jax.numpy as jnp
import os
import haiku as hk
import matplotlib.animation as animation
from utils import load_pkl_dataset
import pickle
import jax
import torch
import haiku as hk
import numpy as np
import jax.numpy as jnp
import optax
import matplotlib.pyplot as plt
from models import q_fn, vae_fn, decode_fn,  get_prior_fn
from functools import partial
from configs.vae_config import get_config as get_vae_config
from configs.bc_config import get_config as get_bc_config
from models import policy_fn
from pathlib import Path
from tensorflow_probability.substrates import jax as tfp
import utils as utils
from flax.training.train_state import TrainState

dist = tfp.distributions
os.environ['CUDA_VISIBLE_DEVICES'] = '6'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# get arbitary config 
bc_config = get_bc_config()
bc_config.env = "MW"
bc_config.env_id = "button-press-v2"

rng_key = jax.random.PRNGKey(0)

data_file = os.path.join(bc_config.data_dir, "sac_mw_button-press-v2_100.pkl")
print(data_file)
# load trained q model

model_base = "/scr/aliang80/changepoint_aug/changepoint_aug/ray_results/6_mw_offline_q_sarsa/train_model_fn_MW_c01b2_00004_4_num_trajs=100_2024-03-07_02-23-57"
model_file = Path(model_base) / "model_ckpts" / "MW_epoch_900.pkl"

with open(model_file, "rb") as f:
    params = pickle.load(f)

print("loading q model from", model_file)

# load dataset
dataset, train_dataloader, test_dataloader, obs_dim, action_dim = (
    load_pkl_dataset(
        data_file,
        batch_size=bc_config.batch_size,
        num_trajs=500,
        train_perc=bc_config.train_perc,
        env="MW"
    )
)

/scr/aliang80/changepoint_aug/changepoint_aug/density_estimation/datasets/sac_mw_button-press-v2_100.pkl
loading q model from /scr/aliang80/changepoint_aug/changepoint_aug/ray_results/6_mw_offline_q_sarsa/train_model_fn_MW_c01b2_00004_4_num_trajs=100_2024-03-07_02-23-57/model_ckpts/MW_epoch_900.pkl
number of rollouts:  100
average length of rollout:  40.94
obs_data shape:  torch.Size([3994, 39]) action_data shape:  torch.Size([3994, 4])
min obs data: torch.return_types.min(
values=tensor([-0.1078,  0.3996,  0.1285,  0.8618, -0.0964,  0.6515,  0.1150,  1.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000, -0.1078,  0.3996,  0.1285,  0.8618, -0.0964,  0.6515,
         0.1150,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000, -0.0964,  0.7501,  0.1150],
       dtype=torch.float64),
indices=tensor([ 107,   76,  873, 2470,   75,  868,    0,    0,    0,    0,    0,    0,
     

In [9]:
start_indices = np.where(dataset[:][-1])[0]
start_indices += 1
start_indices = np.insert(start_indices, 0, 0)
traj_indx = np.random.randint(0, len(start_indices) - 1)

start = start_indices[traj_indx]
end = start_indices[traj_indx + 1]

all_obss, all_actions, all_next_obss, all_next_actions, all_rewards, all_dones = dataset[:]

min_x = torch.min(all_obss[:, 0])
max_x = torch.max(all_obss[:, 0])
min_y = torch.min(all_obss[:, 1])
max_y = torch.max(all_obss[:, 1])

print(start_indices)

obss = all_obss[start:end].numpy()
actions = all_actions[start:end].numpy()
rewards = all_rewards[start:end].numpy()

print(obss.shape, actions.shape, rewards.shape)

T = np.arange(len(obss))
goal = obss[0, -2:]
rng_seq = hk.PRNGSequence(0)
q_fn_apply = partial(
    jax.jit(hk.without_apply_rng(q_fn).apply, static_argnums=(3)),
    hidden_size=bc_config.hidden_size,
)
ts = TrainState.create(
    apply_fn=,
    params=params,
    tx=None,
)
utils.visualize_q_trajectory(ts, next(rng_seq), obss, actions, rewards, goal)

# scat = axes[5].scatter(all_obss[:, 0], all_obss[:, 1], cmap="viridis")

# # compute for each trajectory point with max variance
# print(len(range(len(start_indices) - 1)))
# for i in range(len(start_indices) - 1):
#     start = start_indices[i]
#     end = start_indices[i + 1]
#     obss = all_obss[start:end]

#     var = variance[start:end]
#     # print(var.shape)

#     # top k 
#     k = 1
#     top_k_idx = jnp.argsort(var)[-k:]
#     top_k_points = obss[top_k_idx, :2]
#     axes[5].plot(top_k_points[:, 0], top_k_points[:, 1], "ro", markersize=10)
# axes[5].set_title("Trajectory with max variance")

[   0   36   75  122  160  200  240  282  326  361  399  440  477  513
  551  588  628  667  706  743  783  904  942  979 1018 1059 1096 1132
 1170 1212 1258 1294 1334 1372 1413 1453 1495 1538 1575 1611 1650 1686
 1728 1767 1808 1845 1886 1923 1962 2004 2042 2079 2121 2161 2198 2238
 2276 2316 2354 2395 2436 2474 2514 2551 2588 2626 2668 2708 2748 2790
 2829 2866 2904 2941 2978 3019 3058 3095 3136 3173 3214 3255 3292 3334
 3374 3411 3449 3486 3526 3567 3606 3647 3686 3724 3764 3806 3842 3879
 3914 3955 3994]
(36, 39) (36, 4) (36,)


ValueError: vmap got inconsistent sizes for array axes to be mapped:
  * most axes (4 of them) had size 128, e.g. axis 0 of argument param['q_function/mlp/~/linear_0']['b'] of type float32[128];
  * one axis had size 43: axis 0 of argument param['q_function/mlp/~/linear_0']['w'] of type float32[43,128];
  * one axis had size 1: axis 0 of argument param['q_function/mlp/~/linear_2']['b'] of type float32[1];
  * one axis had size 5: axis 0 of argument rng_key of type uint32[5,2]

In [ ]:
from collections import Counter
start_goal = all_obss[start_indices[:-1], -2:]
print(start_goal.shape)
Counter(start_goal.sum(axis=-1).tolist())
Counter(all_goal_locations.sum(axis=-1).tolist())

(100, 2)


Counter({0.21714340896865436: 999,
         4.308837197806755: 999,
         4.165924230834062: 999,
         1.958908693241645: 999,
         3.6730622004479376: 999,
         3.986351067352034: 999,
         4.135198212828259: 999,
         0.1346858664346735: 999,
         1.9682568325571312: 999,
         4.288305885045938: 600,
         4.410267337047719: 570,
         4.231309871976576: 490,
         3.6882585979758913: 477,
         4.048842200516791: 414,
         3.8733769614702416: 396,
         2.0281928253277424: 308,
         1.8073716783485634: 277,
         2.7781830932591616: 269,
         2.9925080073988672: 269,
         2.076058121131373: 229,
         1.1518863334461238: 223,
         1.6754330461069542: 217,
         1.1455363399461267: 209,
         1.9334255986112687: 209,
         0.6089973690138797: 206,
         0.8471553086870259: 204,
         1.914158999686661: 202,
         -0.3022329229412044: 200,
         -0.1101372449791298: 199,
         1.84286740624

In [51]:
vl_1 = ax1.axvline(x=0)
vl_2 = ax2.axvline(x=0)

def animate(i):
    vl_1.set_xdata([i, i])
    vl_2.set_xdata([i, i])
    traj_plot.set_data(xy[:i, 0], xy[:i, 1])

print("saving anim")
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

ani = animation.FuncAnimation(fig, animate, frames=T, interval=10, blit=True)
ani.save("movie.gif", fps=30, writer='imagemagick')

# %matplotlib notebook
# from IPython.display import HTML
# HTML(ani.to_jshtml())

MovieWriter imagemagick unavailable; using Pillow instead.


saving anim
